![First Power Bi](.\images\dbachecks-logo.png )
# dbachecks - Configurations

You can use dbachecks on the command line as we have seen in the previous notebook but you will not always want to have the default settings for the checks

You can alter any of the configurations using the `Set-DbcConfig` command.

This configuration will be the configuration that is used until `Reset-DbcConfig` is run or the configuration is changed.
This configuration will be used for the user who has run the code.

Again, this notebook requires that you have the containers set up from the Notebook 00 - Setting up the containers for the rest of the containers.ipynb

If you have altered the folder path you will need to alter it below.






# Setting the SqlInstances and ComputerNames

A useful configuration to set is the list of SQL Instances and Computer Names that you want the checks to run against. There are two configurations that will do that.

- app.sqlinstance will set the list of SQL Instances
- app.computername will set the list of ComputerNames

## Setting a configuration

You set a configuration using the `Set-DbcConfig` command. You provide the name (which will auto-complete) and the value

```powershell
Set-DbcConfig -Name ConfigName -Value ConfigValue
```

For the app.sqlinstance you could even set this from your registerd server list or CMS

```powershell
$Production = (Get-DbaRegisteredServer -SqlInstance CMSServer -Group "Production").ServerName
Set-DbcConfig -Name app.sqlinstance -Value $Production
```
You can set the SqlInstance and the ComputerName for the checks in this notebook






In [11]:
# Just in case the config has been altered
$null = Reset-DbcConfig
$SqlInstances = 'localhost,15592','localhost,15593'
Set-DbcConfig -Name app.sqlinstance -Value $SqlInstances
Set-DbcConfig -Name app.computername -Value $SqlInstances
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred


Name             Value                              Description                                                       
----             -----                              -----------                                                       
app.sqlinstance  {localhost,15592, localhost,15593} List of SQL Server instances that SQL-based tests will run against
app.computername {localhost,15592, localhost,15593} List of Windows Servers that Windows-based tests will run against 




and you will no longer need to specify them when calling `Invoke-DbcCheck`

## Invalid Database Owners

You may want to check that databases are not owned by certain accounts.

By default the `InvalidDatabaseOwner` checks that the databases are not owned by the sa account. When we run this check against the containers we pass all of the checks

In [12]:
Invoke-DbcCheck -SqlCredential $SqlCredential  InvalidDatabaseOwner

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15592
      [+] Database AdventureWorks2017 - owner sqladmin should Not be in this list ( sa ) on localhost,15592 2ms
      [+] Database Northwind - owner sqladmin should Not be in this list ( sa ) on localhost,15592 8ms
      [+] Database pubs - owner sqladmin should Not be in this list ( sa ) on localhost,15592 3ms

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15593
      [+] Database AdventureWorks2017 - owner sqladmin should Not be in this list ( sa ) on localhost,15593 4ms
      [+] Database Northwind - owner sqladmin should Not be in this list ( sa ) on localhost,15593 2ms
      [+] Database pubs - owner sqladmin

If our organistation requirements are that no databases should be owned by members of the DBA Team user accounts, we can alter the configuration to check for that. 

To find out which configuration we need to change we can use `Get-DbcCheck`

In [7]:
Get-DbcCheck -Pattern InvalidDatabaseOwner | Format-List



Group       : Database
Type        : Sqlinstance
UniqueTag   : InvalidDatabaseOwner
AllTags     : InvalidDatabaseOwner, Medium, Database
Config      : app.sqlinstance policy.invaliddbowner.name policy.invaliddbowner.excludedb 
Description : Tests that the Database Owner is NOT in the specified (default blank) list





We can see that there are 3 configurations that this check will use
- app.sqlinstance
- policy.invaliddbowner.name 
- policy.invaliddbowner.excludedb 

We can get further information about the checks with `Get-DbcConfig`

In [8]:
Get-DbcConfig -Name policy.invaliddbowner.name | Format-List



Name        : policy.invaliddbowner.name
Value       : sa
Description : The database owner account should not be this user





We can change the value of that configuration item using `Set-DbcConfig`. We will set it to the members of the DBA Team AD Group

This is an example - don't run this unless you are connected to THEBEARD.local domain and your DBAs are in a group called SQL_DBAs_The_Cool_Ones :-)

In [19]:
$DBA_Accounts =  (Get-ADGroup -Identity 'CN=SQL_DBAs_The_Cool_Ones,OU=TheBeardGroups,OU=TheBeard,DC=TheBeard,DC=local' |Get-ADGroupMember).ForEach{"THEBEARD\$($Psitem.SamAccountName)"} 
Set-DbcConfig -Name policy.invaliddbowner.name -Value $DBA_Accounts


Name                       Value                                                                Description            
----                       -----                                                                -----------            
policy.invaliddbowner.name {THEBEARD\Rob, THEBEARD\DBA, THEBEARD\wdurkin, THEBEARD\gsartori...} The database owner a...




# Altering a Configuration
This is how you can alter a check to test the values that YOU want by setting a configuration.  
We have renamed the sa acccount and disabled it in the containers. There is a sysadmin account called `sysadmin` created.  
Perhaps we do not want our databases owned by the new sysadmin account `sqladmin` We can set that value in the configuration and run the test

In [13]:
Set-DbcConfig -Name policy.invaliddbowner.name -Value sqladmin
Invoke-DbcCheck -SqlCredential $SqlCredential -Check InvalidDatabaseOwner


Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15592
      [-] Database AdventureWorks2017 - owner sqladmin should Not be in this list ( sqladmin ) on localhost,15592 3ms
        Expected collection sqladmin to not contain 'sqladmin', because The database owner was one specified as incorrect, but it was found.
        176:                         $psitem.Owner | Should -Not -BeIn $TargetOwner -Because "The database owner was one specified as incorrect"
        at <ScriptBlock>, C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1: line 176
      [-] Database Northwind - owner sqladmin should Not be in this list ( sqladmin ) on localhost,15592 5ms
        Expected collecti

## Running a test, fixing the errors and re-running

Ah

We had better have a word with that fellow Rob and alter all of those database owners!! 

This is how we could ask our junior DBA to do this and test they have done so correctly

We could change the owner with `Get-DbaDatabase -SqlInstance sql2016n1 -ExcludeSystem -Owner THEBEARD\Rob | Set-DbaDbOwner -TargetLogin THEBEARD\DatabaseOwner`

We can also set a confiuguration for our desired database owner and test for that. With dbachecks we can check for both Valid and Invalid database owner accounts.

The code below will 
- Get all databases owned by sysadmin and set the owner to be thebeard
- Run the check to ensure that no databases are owned by sysadmin because we set that configuration above
- Set a configuration for the database owner we expect to thebeard
- Run the check to ensure that the databases are owned by the thebeard



In [14]:
# Set the correct owner
$null = Get-DbaDatabase -SqlInstance $SqlInstances -SqlCredential $SqlCredential -ExcludeSystem -Owner sqladmin | Set-DbaDbOwner -TargetLogin thebeard
# Check that we do not have any databases owned by the unexpected accounts
Invoke-DbcCheck -SqlCredential $SqlCredential -Check InvalidDatabaseOwner
# Set the value for expected database owner
Set-DbcConfig -Name policy.validdbowner.name -Value thebeard
# Check our databases  are owned by the correct account
Invoke-DbcCheck -SqlCredential $SqlCredential -Check ValidDatabaseOwner 

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1' with Tags InvalidDatabaseOwner

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15592
      [+] Database AdventureWorks2017 - owner thebeard should Not be in this list ( sqladmin ) on localhost,15592 1ms
      [+] Database Northwind - owner thebeard should Not be in this list ( sqladmin ) on localhost,15592 2ms
      [+] Database pubs - owner thebeard should Not be in this list ( sqladmin ) on localhost,15592 5ms

  Describing Invalid Database Owner

    Context Testing Database Owners on localhost,15593
      [+] Database AdventureWorks2017 - owner thebeard should Not be in this list ( sqladmin ) on localhost,15593 3ms
      [+] Database Northwind - owner thebeard should Not be in this list ( sqladmin ) on localhost,15593 1ms
      [+] 

## Ensuring Databases exist

You might want to test that a particular database exists on your instance. This could be your DBA database or perhaps you have a check for your companies software system that you want to run on a clients estate and your software requires certain databases to exist 

You can run the default `DatabaseExists` which will check that the system databases exist as you did in the quick examples notebook.

In [15]:
Invoke-DbcCheck -SqlCredential $SqlCredential -Check DatabaseExists

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1' with Tags DatabaseExists

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1

  Describing Database Exists

    Context Database exists on localhost,15592
      [+] Database master should exist on  163ms
      [+] Database msdb should exist on  126ms
      [+] Database tempdb should exist on  126ms
      [+] Database model should exist on  121ms

  Describing Database Exists

    Context Database exists on localhost,15593
      [+] Database master should exist on  117ms
      [+] Database msdb should exist on  122ms
      [+] Database tempdb should exist on  123ms
      [+] Database model should exist on  116ms
Tests completed in 1.72s
Tests Passed: 8, Failed: 0, Skipped: 0, Pending: 0, Inconclusive: 0 


## Finding the configurations for the check

You can find the configurations that you can set for a check with `Get-DbcCheck`

In [16]:
Get-DbcCheck DatabaseExists | Format-List



Group       : Database
Type        : Sqlinstance
UniqueTag   : DatabaseExists
AllTags     : DatabaseExists, Database
Config      : app.sqlinstance database.exists 
Description : Tests that the databases are specified are on the instance (Note - Does not check if they are available 
              - Use DatabaseStatus for that))






## Setting the configuration for a check
You can see that you can set the configuration `database.exists` Note that we are using the `-Append` parameter to add to the configuration instead of replacing it.

In [17]:
Set-DbcConfig -Name database.exists -Value 'pubs','NorthWind','AdventureWorks2017','WideWorldImporters' -Append


Name            Value                            Description                                   
----            -----                            -----------                                   
database.exists {master, msdb, tempdb, model...} The databases we expect to be on the instances




## Run the check with the new configuration
and then we can run the check again and confirm that we have all of the expected databases.

In [18]:
Invoke-DbcCheck -SqlCredential $SqlCredential -Check DatabaseExists


Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1' with Tags DatabaseExists

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Database.Tests.ps1

  Describing Database Exists

    Context Database exists on localhost,15592
      [+] Database master should exist on  119ms
      [+] Database msdb should exist on  116ms
      [+] Database tempdb should exist on  114ms
      [+] Database model should exist on  162ms
      [+] Database pubs should exist on  153ms
      [+] Database NorthWind should exist on  149ms
      [+] Database AdventureWorks2017 should exist on  174ms
      [-] Database WideWorldImporters should exist on  360ms
        Expected 'WideWorldImporters' to be found in collection @('AdventureWorks2017', 'master', 'model', 'msdb', 'Northwind', 'pubs', 'tempdb'), because We expect WideWorldImporters to be on localhost,15592, but it was not found.
        70:     $Actual | Should -C

## Ensuring the Agent Alerts are as expected

You might want to check that you have all of the expected Agent Alerts. By default dbachecks will check for alerts for Severity 16-25 and messageid 823,824,825

In [19]:
Invoke-DbcCheck -SqlCredential $SqlCredential -Check AgentAlert

Executing all tests in 'C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1' with Tags AgentAlert

Executing script C:\Program Files\WindowsPowerShell\Modules\dbachecks\1.2.15\checks\Agent.Tests.ps1

  Describing Agent Alerts

    Context Testing Agent Alerts Severity exists on localhost,15592
      [+] localhost,15592 should have Severity 16 Alert 148ms
      [+] localhost,15592 should have Severity 16 Alert enabled 5ms
      [+] localhost,15592 should have notification for Severity 16 Alert 10ms
      [+] localhost,15592 should have Severity 17 Alert 3ms
      [+] localhost,15592 should have Severity 17 Alert enabled 3ms
      [+] localhost,15592 should have notification for Severity 17 Alert 4ms
      [+] localhost,15592 should have Severity 18 Alert 3ms
      [+] localhost,15592 should have Severity 18 Alert enabled 3ms
      [+] localhost,15592 should have notification for Severity 18 Alert 3ms
      [+] localhost,15592 should have Severity 19 Alert 

## Get the configuration items available for the check
You can check the configuration items available for the check

In [20]:
Get-DbcCheck -Pattern AgentAlert | Format-List



Group       : Agent
Type        : Sqlinstance
UniqueTag   : AgentAlert
AllTags     : AgentAlert, Agent
Config      : app.sqlinstance agent.alert.Severity agent.alert.messageid agent.alert.Job agent.alert.Notification 
Description : Tests that there are Agent Alerts set up for the specified (default 16-25) alert severities and ids 
              (default 823-825) and if specified Agent Jobs and/or notifications





The configuration items available are
- app.sqlinstance 
- agent.alert.Severity 
- agent.alert.messageid 
- agent.alert.Job 
- agent.alert.Notification

(Until Rob fixes the code) You can get the configuration values and descriptions for the configuration items

In [ ]:
'app.sqlinstance', 'agent.alert.Severity', 'agent.alert.messageid', 'agent.alert.Job', 'agent.alert.Notification' | ForEach-Object {
    Get-DbcConfig -Name $PSItem | Format-List
}

If you use all of the TigerTeams Agent Alerts then you might want to check that they are installed and enabled. You can alter the configuration to do this as follows. This time we have not used the append parameter and have replaced all of the values.

In [39]:
$Ids = '1101','1105','1121','1214','17130','17179','17300','17883','17884','17887','17888','17890','2508','2511','28036','3271','3452','3619','3624','5180','5228','5229','5242','5243','5250','5572','5901','605','701','802','823','824','825','832','833','845','855','856','8966','9002','9100'
Set-DbcConfig -Name agent.alert.messageid -Value $Ids

and then you can check for those as well

In [40]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbachecks'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Invoke-DbcCheck -SqlCredential $SqlCredential -Check AgentAlert